In [1]:
! pip3 install --upgrade --user google-cloud-aiplatform==1.60.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 60.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [86]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

qwiklabs-gcp-03-480091a802dc


In [2]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [10]:
# TODO: Define a function to add two numerical inputs and return the result.
# Keep the print statement within the function.

def add_numbers(a, b):
    print("Calling add function")
    return a + b

# TODO: Define a function to multiply two numerical inputs and return the result.
# Keep the print statement within the function.

def multiply_numbers(a, b):
    print("Calling multiply function")
    return a * b

# TODO: Create FunctionDeclarations for your functions

add_function_declaration = FunctionDeclaration(
    name="add_numbers",
    description="Adds two numbers together.",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "The first number to add."},
            "b": {"type": "number", "description": "The second number to add."}
        },
        "required": ["a", "b"]
    }
)

multiply_function_declaration = FunctionDeclaration(
    name="multiply_numbers",
    description="Multiplies two numbers together.",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "The first number to multiply."},
            "b": {"type": "number", "description": "The second number to multiply."}
        },
        "required": ["a", "b"]
    }
)


In [11]:
# add_function_declaration = FunctionDeclaration.from_func(add_numbers)

# multiply_function_declaration = FunctionDeclaration.from_func(multiply_numbers)

In [12]:
math_tool = Tool(
    function_declarations=[
        add_function_declaration,
        multiply_function_declaration
    ],
)

In [13]:
model = GenerativeModel(
    model_name="gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[math_tool],
)

In [14]:
# Define the user's prompt in a Content object that we can reuse in model calls
user_prompt_content = Content(
    role="user",
    parts=[
        Part.from_text("""
    You are a helpful assistant.

    - Fulfill the user's instructions.
    - If asked to add or multiply numbers, call the provided functions.
    - You may call one function after the other if needed.
    - Repeat the result to the user.
    """),
    ],
)

In [15]:
chat = model.start_chat()

In [16]:
response = chat.send_message("multiply 7 and 8")

In [25]:
response = chat.send_message("""
    You are a helpful assistant.

    - Fulfill the user's instructions.
    - If asked to add or multiply numbers, call the provided functions.
    - You may call one function after the other if needed.
    - Repeat the result to the user.
    """)

In [26]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Sounds like a plan! I\'m ready to assist you with adding and multiplying numbers. Just let me know what you need. \360\237\230\212 \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.07470703125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.06640625
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.1396484375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.119140625
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.0673828125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.053466796875
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.08056640625
    severity: HARM_SEVERITY_NEGLI

In [27]:
def handle_response(response):
    # If there is a function call then invoke it
    # Otherwise, print the response.
    if response.candidates[0].function_calls:
        function_call = response.candidates[0].function_calls[0]
    else:
        print(response.text)
        return
    
    # Extract the function name
    function_name = function_call.name
    
    # Complete the following sections
    if function_name == "add_numbers":
        # Extract the arguments to use in your function
        arguments = function_call.args
        # Assuming arguments are in JSON string format
        args = arguments
        a = args.get("a")
        b = args.get("b")
        # Call your function
        result = add_numbers(a, b)
        # Send the result back to the chat session with the model
        new_response = chat.send_message(str(result))
        # Make a recursive call of this handler function
        handle_response(new_response)
    
    elif function_name == "multiply_numbers":
        # Extract the arguments to use in your function
        arguments = function_call.args
        # Assuming arguments are in JSON string format
        args = arguments
        a = args.get("a")
        b = args.get("b")
        # Call your function
        result = multiply_numbers(a, b)
        # Send the result back to the chat session with the model
        new_response = chat.send_message(str(result))
        # Make a recursive call of this handler function
        handle_response(new_response)

    else:
        # You shouldn't end up here
        print(f"Unexpected function call: {function_call.name}")


In [28]:
handle_response(response)

Sounds like a plan! I'm ready to assist you with adding and multiplying numbers. Just let me know what you need. 😊 



In [29]:
response = chat.send_message("Tell me a joke.")
handle_response(response)

Why don't they play poker in the jungle? 

Too many cheetahs! 😉 



In [30]:
response = chat.send_message("I have 7 pizzas each with 16 slices. How many slices do I have?")
handle_response(response)

Calling multiply function
You have 112.0 slices. 



In [31]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. How many pizzas did they bring together?")
handle_response(response)

Calling add function
They brought 7.0 pizzas together. 



In [32]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. There are 16 slices per pizza. How many slices are there?")
handle_response(response)    

Calling add function
Calling multiply function
There are 112.0 slices. 



In [33]:
response = chat.send_message("Doug brought 4 pizzas, but Andrew dropped 2 on the ground. How many pizzas are left?")
handle_response(response)  

Calling add function
There are 2.0 pizzas left. 

